In [1]:
#import sys
#sys.path.append("../")

In [1]:
from dotenv import load_dotenv
from langchain.embeddings import OpenAIEmbeddings
from psycopg.conninfo import make_conninfo

import os
import psycopg

load_dotenv()

os.environ["OPEN_API_KEY"] = os.getenv('OPEN_API_KEY')

embeddings = OpenAIEmbeddings(openai_api_key=os.environ["OPEN_API_KEY"])

connectionString = make_conninfo(
  host=os.getenv('DB_HOST'),
  port=os.getenv('DB_PORT'),
  dbname=os.getenv('DB_DATABASE'),
  user=os.getenv('DB_USER'),
  password=os.getenv('DB_PASSWORD')
)

In [3]:

# Load sample documents into the database

directory = '../data'

# Loop over all files in the directory, create embeddings, save them to the database
for file_name in os.listdir(directory):
  file_path = os.path.join(directory, file_name)
  with open(file_path, 'r') as file:
      content = file.read()
      embedding_results = embeddings.embed_query(content)
      with psycopg.connect(connectionString) as conn:
        insert_query = "INSERT INTO documents (file_name, content, embedding) VALUES (%s, %s, %s)"
        insert_data = (file_name, content, embedding_results)
        conn.execute(insert_query, insert_data)
        conn.commit()


PermissionError: [Errno 13] Permission denied: '../data\\lanschool'

In [17]:
search_text = "what if teacher forget to end the class"

In [18]:
# embed search text
embedding_results = embeddings.embed_query(search_text)

In [21]:
# find documents based on cosin similarity to the embedded search text
with psycopg.connect(connectionString) as conn: 
  select_query = "SELECT match_documents(%s,%s,%s)"
  query_data = (f'{embedding_results}', 0.8, 10)
  results = conn.execute(select_query, query_data).fetchall()
  print(results)
  for result in results:
    print(result)  

[]
